In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
weather = pd.read_csv(
    "/Users/ishitajain/5c_data/5c-Datathon-2025/data_and_starter_code/data/historical_weather.csv"
)

In [ ]:
produc = pd.read_csv("baltic_energy_type.csv")

In [ ]:
solar_produc = produc[produc["Production Type"] == "Solar"]

We are using the PVwatts thiing

it require poa irradianvce 
dependning on the irradiance, there are 2 differnet forms/ 


In [ ]:
solar_produc = solar_produc[solar_produc["Area"] == "Estonia"]

In [ ]:
weather

In [ ]:
weather["date"] = pd.to_datetime(weather["datetime"])

In [ ]:
solar_produc["date"] = pd.to_datetime(solar_produc["MTU (CET/CEST)"], format="%d/%m/%Y")

In [ ]:
weather.columns

In [ ]:
weather["date"] = pd.to_datetime(weather["date"])

In [ ]:
print(weather.head())

In [ ]:
import pandas as pd
import numpy as np

# Convert datetime and set as index


# Helper for RMS (root mean square)
def rms(x):
    return np.sqrt((x**2).mean())


# Helper for circular mean in degrees
def circular_mean_deg(deg_series):
    radians = np.deg2rad(deg_series)
    sin_mean = np.mean(np.sin(radians))
    cos_mean = np.mean(np.cos(radians))
    return np.rad2deg(np.arctan2(sin_mean, cos_mean)) % 360


# Define aggregation functions
agg_funcs = {
    "temperature": "mean",
    "dewpoint": "mean",
    "rain": "sum",
    "snowfall": "sum",
    "surface_pressure": "mean",
    "cloudcover_total": "mean",
    "cloudcover_low": "mean",
    "cloudcover_mid": "mean",
    "cloudcover_high": "mean",
    "windspeed_10m": rms,
    "winddirection_10m": circular_mean_deg,
    "shortwave_radiation": "mean",
    "direct_solar_radiation": "mean",
    "diffuse_radiation": "mean",
    "latitude": "mean",
    "longitude": "mean",
}

# Apply daily resampling
# Assuming you already have the weather DataFrame

# Convert datetime column to datetime format and set as index
weather["datetime"] = pd.to_datetime(weather["datetime"])
weather = weather.set_index("datetime")

# Now resample daily and aggregate
daily_df = weather.resample("D").agg(agg_funcs).reset_index()

In [ ]:
daily_df

In [ ]:
daily_df["precipitation"] = daily_df["rain"] + daily_df["snowfall"]
daily_df.drop(
    ["rain", "snowfall", "cloudcover_low", "cloudcover_mid", "cloudcover_high"],
    axis=1,
    inplace=True,
)

In [ ]:
daily_df = daily_df.rename({"datetime": "date"})

In [ ]:
solar_produc

In [ ]:
daily_df.rename({"datetime": "date"}, axis=1, inplace=True)

In [ ]:
daily_df

In [ ]:
# Filter solar_produc for Estonia


# Merge the filtered solar_produc with weather on the date column
merged_data = pd.merge(solar_produc, daily_df, on="date")

# Display the merged data
merged_data

In [ ]:
merged_data.drop(["MTU (CET/CEST)", "Area", "Production Type"], axis=1, inplace=True)

In [ ]:
merged_data.columns

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(merged_data.corr())

In [ ]:
#

In [ ]:
# import numpy as np

# merged_data["log_generation"] = np.log1p(
#     merged_data["Generation (MW)"] + 1
# )  # log(1 + x) to handle 0s

In [ ]:
merged_data.drop(
    ["dewpoint", "shortwave_radiation", "diffuse_radiation", "latitude", "longitude"],
    axis=1,
    inplace=True,
)

In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# # Drop date column and isolate predictors and target
# drop_cols = ["date"]
# target = "Generation (MW)"
# predictors = [col for col in df.columns if col not in [target] + drop_cols]

# # Standardize predictors
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df[predictors])
# y = df[target]

# # Fit linear regression
# linreg = LinearRegression()
# linreg.fit(X_scaled, y)

# # Get feature importance (absolute coefficient values)
# importance = np.abs(linreg.coef_)

# # Create a DataFrame for easy plotting
# importance_df = pd.DataFrame(
#     {"Feature": predictors, "Importance": importance}
# ).sort_values(by="Importance", ascending=False)

# # Plot
# plt.figure(figsize=(10, 6))
# sns.barplot(x="Importance", y="Feature", data=importance_df, palette="viridis")
# plt.title("Feature Importance (Linear Regression Coefficients)")
# plt.xlabel("Absolute Coefficient Value")
# plt.ylabel("Feature")
# plt.tight_layout()
# plt.show()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # For each predictor
# for col in [
#     "dewpoint",
#     "surface_pressure",
#     "cloudcover_total",
#     "shortwave_radiation",
#     "direct_solar_radiation",
#     "diffuse_radiation",
#     "precipitation",
# ]:
#     sns.scatterplot(x=df[col], y=df["Generation (MW)"])
#     plt.title(f"{col} vs Generation (MW)")
#     plt.show()

In [ ]:
# additional_columns_to_remove = [
#     "windspeed_10m",
#     "winddirection_10m",
#     "day_of_year",å
#     "temperature",
#     "day_of_week",
#     "month",
#     "latitude",
#     "longitude",
#     "hour",
# ]


# # Function to remove these columns from your existing dataframe
# def remove_columns(df, columns_to_remove):
#     for col in columns_to_remove:
#         if col in df.columns:
#             df = df.drop(col, axis=1)
#             print(f"Removed column: {col}")
#     return df


# merged_data = remove_columns(merged_data, additional_columns_to_remove)

In [ ]:
import matplotlib.pyplot as plt

# Plot direct solar radiation over time
plt.figure(figsize=(12, 6))
plt.scatter(
    merged_data["date"],
    merged_data["direct_solar_radiation"],
    label="Direct Solar Radiation",
    color="orange",
)
plt.xlabel("Date")
plt.ylabel("Direct Solar Radiation")
plt.title("Direct Solar Radiation Over Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
df = merged_data.copy()

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import pandas as pd

# # VIF dataframe
# X_no_const = df[predictors]
# vif_data = pd.DataFrame()
# vif_data["feature"] = X_no_const.columns
# vif_data["VIF"] = [
#     variance_inflation_factor(X_no_const.values, i) for i in range(X_no_const.shape[1])
# ]
# print(vif_data)

In [ ]:
df

In [ ]:
merged_data.columns

In [ ]:
merged_data

In [ ]:
merged_data

In [ ]:
import numpy as np

merged_data["Generation_log"] = np.log(
    merged_data["Generation (MW)"] + 1
)  # Add 1 to avoid log(0)

In [ ]:
import statsmodels.api as sm

# Define X and y
X = merged_data.drop(
    ["Generation (MW)", "Generation_log", "date"], axis=1
)  # example predictors
X = sm.add_constant(X)  # adds intercept
y = merged_data["Generation_log"]  # use original or transformed y

model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Add constant to the dataset (for the intercept in VIF calculation)
X_with_const = add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_with_const.values, i)
    for i in range(X_with_const.shape[1])
]

# Sort by VIF
vif_data = vif_data.sort_values(by="VIF", ascending=False)

print(vif_data)

In [ ]:
merged_data["output_one_pan"] = 1.63 * (merged_data["direct_solar_radiation"]) * 0.2

In [ ]:
merged_data

In [ ]:
import matplotlib.pyplot as plt

# Plot cloud cover over time
plt.figure(figsize=(12, 6))
plt.scatter(
    merged_data["date"],
    merged_data["cloudcover_total"],
    label="Cloud Cover",
    color="blue",
)
plt.xlabel("Date")
plt.ylabel("Cloud Cover (%)")
plt.title("Cloud Cover Over Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
numeric_cols = [
    "Generation (MW)",
    "temperature",
    "surface_pressure",
    "cloudcover_total",
    "windspeed_10m",
    "direct_solar_radiation",
    "precipitation",
]

In [ ]:
merged_data

In [ ]:
import matplotlib.pyplot as plt

# Plot direct solar radiation over time
plt.figure(figsize=(12, 6))
plt.scatter(
    merged_data["date"],
    merged_data["precipitation"],
    label="Direct Solar Radiation",
    color="orange",
)
plt.xlabel("Date")
plt.ylabel("Direct Solar Radiation")
plt.title("Direct Solar Radiation Over Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
merged_data

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

X = merged_data[numeric_cols].drop(["Generation (MW)"], axis=1)  # drop target variable

# Add constant
X_const = add_constant(X)

# Compute VIF
vif_df = pd.DataFrame()
vif_df["Feature"] = X.columns
vif_df["VIF"] = [
    variance_inflation_factor(X_const.values, i + 1) for i in range(len(X.columns))
]  # skip constant

# Sort by VIF descending
vif_df = vif_df.sort_values(by="VIF", ascending=False)
print(vif_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Assuming merged_Data is your dataset
# Separate the features (X) and target variable (y)
X = merged_data[["temperature", "cloudcover_total", "direct_solar_radiation"]]
# Replace 'target_column_name' with your target variable
y = merged_data[
    "Generation (MW)"
]  # Replace 'target_column_name' with your target variable

In [ ]:
y

In [ ]:
import statsmodels.api as sm

# Define features and target

# Add constant (intercept)
X = sm.add_constant(X)

# Fit OLS model
lin_model = sm.OLS(y, X).fit()

# Show full summary
print(lin_model.summary())

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Assume merged_data is your DataFrame
# It must have a datetime column and the target column

df = merged_data[["date", "direct_solar_radiation"]].copy()
df.rename(columns={"date": "ds", "direct_solar_radiation": "y"}, inplace=True)

model1 = Prophet(
    growth="flat",  # allow a gentle long-term trend
    yearly_seasonality=True,
    daily_seasonality=True,
    weekly_seasonality=False,  # optional for temp
    seasonality_mode="additive",  # small change over time → additive is safer
)

model1.fit(df)
# Forecast future
future1 = model1.make_future_dataframe(periods=2500, freq="D")  # change as needed
forecast1 = model1.predict(future1)
import numpy as np

# Add stochastic component (volatility) to represent radiation variability
# Set seed for reproducibility
np.random.seed(42)

# Define noise level — adjust this based on observed variation (e.g., ±50 W/m²)
volatility_std = 5  # Change this depending on typical solar radiation variance

# Limit to 5 years
end_date = df["ds"].min() + pd.DateOffset(years=9)
forecast_subset_1 = forecast1[forecast1["ds"] <= end_date].copy()

# Add random noise to yhat predictions
forecast_subset_1["yhat_stochastic"] = forecast_subset_1["yhat"] + np.random.normal(
    loc=0, scale=volatility_std, size=len(forecast_subset_1)
)

# Plot deterministic vs. stochastic predictions
plt.figure(figsize=(12, 6))
plt.plot(
    forecast_subset_1["ds"],
    forecast_subset_1["yhat"],
    label="Deterministic Forecast",
    alpha=0.8,
)
plt.plot(
    forecast_subset_1["ds"],
    forecast_subset_1["yhat_stochastic"],
    label="Stochastic Forecast",
    linestyle="--",
    alpha=0.7,
)
plt.xlabel("Date")
plt.ylabel("Direct Solar Radiation (W/m²)")
plt.title("Prophet Forecast of Solar Radiation with Stochastic Noise")
plt.legend()
plt.grid(True)
plt.show()


# Plot
model1.plot(forecast1)
plt.title("Forecast of Cloud Cover")
plt.xlabel("Time")
plt.ylabel("Cloud Cover Forecast")
plt.show()

# Optional: seasonal components
model1.plot_components(forecast1)
plt.show()

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np

# Prep data
df = merged_data[["date", "cloudcover_total"]].copy()
df.rename(columns={"date": "ds", "cloudcover_total": "y"}, inplace=True)

# Fit Prophet model
model2 = Prophet(
    growth="flat",
    yearly_seasonality=True,
    daily_seasonality=True,
    weekly_seasonality=False,
    seasonality_mode="additive",
)

model2.fit(df)

# Forecast
future2 = model2.make_future_dataframe(periods=2500, freq="D")
forecast2 = model2.predict(future2)

# --------------------------------------
# ✅ Add stochastic noise
# Estimate residual std dev from training data
train_forecast = model2.predict(df)
residuals = df["y"] - train_forecast["yhat"]
residual_std = residuals.std()

# Add random noise to simulate variation
np.random.seed(42)  # for reproducibility
forecast2["yhat_stochastic"] = forecast2["yhat"] + np.random.normal(
    0, residual_std * 0.3, size=len(forecast2)
)
# Optional: clip to 0–100% cloud cover if necessary
forecast2["yhat_stochastic"] = forecast2["yhat_stochastic"].clip(0, 100)

# --------------------------------------

In [ ]:
import matplotlib.pyplot as plt

# Plot deterministic vs stochastic forecast
plt.figure(figsize=(14, 6))

# Original Prophet forecast
plt.plot(
    forecast2["ds"],
    forecast2["yhat"],
    label="Deterministic Forecast",
    color="blue",
    alpha=0.5,
)

# Stochastic version (with noise added)
plt.plot(
    forecast2["ds"],
    forecast2["yhat_stochastic"],
    label="Stochastic Forecast",
    color="orange",
    linewidth=1,
)

# Optional: highlight the historical part
plt.axvline(
    x=df["ds"].max(), color="gray", linestyle="--", alpha=0.6, label="Forecast Start"
)

# Labels and title
plt.title("Cloud Cover Forecast in Estonia (with Stochasticity)", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Cloud Cover (%)", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np

# Prepare data
df = merged_data[["date", "temperature"]].copy()
df.rename(columns={"date": "ds", "temperature": "y"}, inplace=True)

# Add time-based regressor: 0.02°C per year
df["t"] = (df["ds"] - df["ds"].min()) / np.timedelta64(1, "Y")
df["linear_trend"] = df["t"] * 0.02  # Adjust slope here if needed

# Initialize model
model3 = Prophet(
    growth="flat",  # We'll use our custom trend instead
    yearly_seasonality=True,
    changepoint_prior_scale=0.001,
    daily_seasonality=True,
    weekly_seasonality=False,
    seasonality_mode="additive",
)

# Add custom regressor
model3.add_regressor("linear_trend")

# Fit the model
model3.fit(df[["ds", "y", "linear_trend"]])

# Make future dataframe
future3 = model3.make_future_dataframe(periods=2500, freq="D")

# Add the same regressor to future
future3["t"] = (future3["ds"] - df["ds"].min()) / np.timedelta64(1, "Y")
future3["linear_trend"] = future3["t"] * 0.02

# Predict
forecast3 = model3.predict(future3)
# Add a stochastic component (volatility)
# Define volatility — e.g., standard deviation = 1.0°C for Estonia
np.random.seed(42)  # For reproducibility
volatility_std = 1.0  # Adjust this based on how volatile you expect

# Filter forecast for next 5 years
end_date = df["ds"].min() + np.timedelta64(9, "Y")
forecast_subset_3 = forecast3[forecast3["ds"] <= end_date].copy()

# Add stochastic noise to yhat (predicted value)
forecast_subset_3["yhat_stochastic"] = forecast_subset_3["yhat"] + np.random.normal(
    0, volatility_std, size=len(forecast_subset_3)
)

# Plot deterministic vs. stochastic forecast
plt.figure(figsize=(12, 6))
plt.plot(
    forecast_subset_3["ds"],
    forecast_subset_3["yhat"],
    label="Deterministic Forecast",
    alpha=0.8,
)
plt.plot(
    forecast_subset_3["ds"],
    forecast_subset_3["yhat_stochastic"],
    label="Stochastic Forecast",
    linestyle="--",
    alpha=0.7,
)
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.title("Prophet Forecast with +0.02°C/year Trend and Stochastic Noise")
plt.legend()
plt.grid(True)
plt.show()

# Plot forecast
model3.plot(forecast3)
plt.title("Forecast of Temperature with +0.02°C/year Trend")
plt.xlabel("Time")
plt.ylabel("Temperature Forecast")
plt.show()

# Plot seasonal components
model3.plot_components(forecast3)
plt.show()

In [ ]:
# Forecast 1: Solar Radiation (use yhat_stochastic if you added noise, otherwise yhat)
forecast_radiation = forecast_subset_1[["ds", "yhat_stochastic"]].copy()
forecast_radiation.rename(
    columns={"yhat_stochastic": "direct_solar_radiation"}, inplace=True
)

# Forecast 2: Cloud Cover — must use yhat_stochastic (assuming you've already added it)
forecast_cloud = forecast2[["ds", "yhat_stochastic"]].copy()
forecast_cloud.rename(columns={"yhat_stochastic": "cloudcover_total"}, inplace=True)

# Forecast 3: Temperature — use yhat_stochastic if you added it, or stick with yhat
forecast_temp = forecast_subset_3[["ds", "yhat_stochastic"]].copy()
forecast_temp.rename(columns={"yhat_stochastic": "temperature"}, inplace=True)

In [ ]:
forecast_subset_1

In [ ]:
forecast2

In [ ]:
forecast_subset_3

In [ ]:
# Merge all three on 'ds'
forecast_all = forecast_temp.merge(forecast_radiation, on="ds", how="inner")
forecast_all = forecast_all.merge(forecast_cloud, on="ds", how="inner")

In [ ]:
forecast_all

In [ ]:
forecast_all = forecast_all[forecast_all["ds"] >= "2023-05-30"]

In [ ]:
merged_data

In [ ]:
forecast_all["const"] = 1.0

In [ ]:
lin_model.summary()

In [ ]:
forecast_all["Generation (MW)"] = lin_model.predict(forecast_all.drop(["ds"], axis=1))

In [ ]:
forecast_all

In [ ]:
X

In [ ]:
X["Generation (MW)"] = y

In [ ]:
X["ds"] = merged_data["date"]

In [ ]:
df = pd.concat([X, forecast_all])

In [ ]:
df

In [ ]:
forecast_all.to_csv("forecasted_consumption.csv")

In [ ]:
# forecast_all["Generation (MW)"] = np.expm1(forecast_all["Generation (MW)"])

In [ ]:
forecast_all["Generation (MW)"] = forecast_all["Generation (MW)"].apply(
    lambda x: max(x, 0)
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(
    forecast_all["ds"],
    forecast_all["Generation (MW)"],
    label="Generation (MW)",
    color="blue",
)
plt.xlabel("Date")
plt.ylabel("Generation (MW)")
plt.title("Generation (MW) Over Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot Generation (MW) over time
plt.figure(figsize=(12, 6))
plt.scatter(
    merged_data["date"],
    merged_data["Generation (MW)"],
    label="Generation (MW)",
    color="blue",
)
plt.xlabel("Date")
plt.ylabel("Generation (MW)")
plt.title("Generation Over Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(df["ds"], df["Generation (MW)"])

In [ ]:
x = merged_data["Generation (MW)"] / merged_data["output_one_pan"]

In [ ]:
y = merged_data["date"]

In [ ]:
df = pd.DataFrame({"Date": y, "Value": x})

In [ ]:
df["Date_ordinal"] = df["Date"].map(pd.Timestamp.toordinal)

# Step 3: Detect outliers in Value using IQR
Q1 = df["Value"].quantile(0.25)
Q3 = df["Value"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_clean = df[(df["Value"] >= lower_bound) & (df["Value"] <= upper_bound)]

# Step 4: Regression on cleaned data
X = df_clean[["Date_ordinal"]]
y = df_clean["Value"]

model = LinearRegression()
model.fit(X, y)

# Optional: plot
plt.scatter(df_clean["Date"], y, label="Cleaned data")
plt.plot(df_clean["Date"], model.predict(X), color="red", label="Regression line")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
import statsmodels.api as sm

In [ ]:
X = df_clean["Date_ordinal"]
X = sm.add_constant(X)  # Add intercept
y = df_clean["Value"]
model = sm.OLS(y, X).fit()

# Step 4: Confidence intervals
print(model.summary())
print("\nConfidence Intervals (95% default):")
print(model.conf_int())

# Optional: Predict and plot with CI
pred = model.get_prediction(X)
pred_summary = pred.summary_frame(alpha=0.05)  # 95% CI

plt.figure(figsize=(10, 6))
plt.scatter(df_clean["Date"], y, label="Data")
plt.plot(df_clean["Date"], pred_summary["mean"], color="red", label="Regression line")
plt.fill_between(
    df_clean["Date"],
    pred_summary["mean_ci_lower"],
    pred_summary["mean_ci_upper"],
    color="red",
    alpha=0.2,
    label="95% CI",
)
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.title("Linear Regression with 95% Confidence Interval")
plt.show()

In [ ]:
# Step 5: Predict today's value
from datetime import date

today = pd.to_datetime(date.today())
today_ordinal = today.toordinal()

# Manually construct with const and Date_ordinal
X_today = pd.DataFrame(
    {"const": [1], "Date_ordinal": [today_ordinal]}  # Intercept term
)

# Predict with confidence interval
pred_today = model.get_prediction(X_today)
summary_today = pred_today.summary_frame(alpha=0.05)  # 95% CI

# Print prediction and CI
mean = summary_today["mean"].values[0]
lower = summary_today["mean_ci_lower"].values[0]
upper = summary_today["mean_ci_upper"].values[0]

print(f"Predicted value for {today.date()}: {mean:.2f}")
print(f"95% Confidence Interval: [{lower:.2f}, {upper:.2f}]")

In [ ]:
y = df_clean["Date"].map(pd.Timestamp.toordinal).values.reshape(-1, 1)

In [ ]:
print(df.tail())

In [ ]:
model = LinearRegression().fit(y, df["Value"])

In [ ]:
model.intercept_

In [ ]:
plt.scatter(y, x)